In [95]:
import tensorflow as tf
import re

seq2seq 模型最开始应用于机器翻译。

机器翻译
早期的机器翻译的思路十分简单，通过设置大量的翻译规则，构建一个大型的双语对照表，来将源语言翻译成目标语言。

后来1990-2010s, 研究者梦开发了更先进复杂的机器翻译技术，（统计机器翻译）Statistical Machine Translation, SMT.的主要原理是从大量的数据中学习一个概率模型p(y|x), 其中x是源语言，y是目标语言。翻译的时候，需要通过求argmaxyP(y|x) 就行

y和x 都是一个句子。

$$P(y|x)=\frac{P(y)P(x|y)}{P(x)}$$

那么
$$argmax_yP(y|x)=argmax_y\frac{P(y)P(x|y)}{P(x)}=argmax_yP(y)P(x|y)$$

P(y)是一个语言模型，而P(x|y)则被称为翻译模型。LM通过目标语料库进行训练，而TM通过平行语料进行训练。

在学习完LM和TM这两个模型之后，需要使用模型，即寻找最佳翻译，y的过程。这个过程也被称为decoding

decoding的做法是通过CRF或HMM的解码。统计机器翻译，虽然比较强但是存在一些问题
- SMT的门槛比较高，表现比较好的模型，通常比较复杂
- SMT模型需要庞大的人力去维护

神经机器翻译（NMT）
NMT的优势在于将大量的人工操作进行省略，下面是SMT和NMT的对比
![title](img/SMT-vs-NMT.png)
NMT 使用网络结构是sequence-to-sequence, 即seq2seq。 同时也被称为encoder-decoder结构。


![title](img/seq2seq_1.png)
根据图中结构，seq2seq中的Encoder读取输入文本，也就是源语言文本。encoder的作用是将输入文本转化成向量，Decoder的作用是将向量转化为输出文本，即目标语言文本。Encoder的行为是编码，而Decoder的行为是解码。

对于其中的具体结构，如下所示

在Encoder短，将source通过Embedding层，将文本转化为词向量，然后经过一个RNN\LSTM\GRU等RNN类型网络层，输出背景向量（context vector）

在Decoder端， 输入的是背景向量以及目标语言的词向量。目标语言的第一个输入是<start>, 每一步根据当前正确的输入词以及上一步的隐状态来预处下一步的输出词。
    
   
 对于预测的时候，Encoder端和训练时相同，而Decoder端部分需要Context vector 以及 初始输入"<start>"。然后后面的RNN结构的网络层的每一次的输入是上一个单元的隐藏层，以及预测的本次的单词的词向量。直到预测的单词为"<end>"或者序列长度达到指定值为止。
 
decoder在训练的时候和预测的时候，其流程有所不同，这两个模式有专门的名词。根据标准答案也就训练的方式，称为（teacher forcing）, 根据上一步的输出作为下一步输入的decode方式为（free running）. 也就是预测的时候用。
    
free running 在训练的时候可以用，这在理论上没问题，可是，在实践中，由于没有指导，那就会导致误差爆炸（bias exposure）, 就是说前面一步错了，后面错的可能性就会更高，所谓一步错，步步错。但是如果在每一步预测的时候，给予相应的指导，那么decoder就可以让训练更快收敛。
    
然而，teacher forcing的方式同样存在一些问题，就是预测时，没有了指导（标记），那么同样有可能会偏离正确的道路。
    
 比较好的思路，就将两种方法进行结合，也就是，既有指导，也有自由发挥。具体的实现方式，就是我们设置一个概率p, 随机一个0-1的值。如果大于p，那么就使用靠上一步的输入来预测，反之，则使用指导来进行预测，这种方法称为计划采样（scheduled sampling）
  
## seq2seq 的损失函数
    
 $$J=-log(p(\widehat{y_1}))-log(p(\widehat{y_2}))-...-log(p([EOS]))=\frac{1}{T}\sum_{i}^{T}{log(p(\widehat(y_i)))}$$
 其中T代表Decode有多少步，[EOS] 表示 end of sentence. 
    
## Decoding 和Bean Search
    
对于最后预测结果的产生，有很多种方式，包括greedy算法、全局搜索以及beam 搜索。全局搜索的计算复杂度指数型的，greedy算法虽然是线性的，但是很难达到全局最优，甚至次优都可能达不到。beam搜索的方法相对于greedy方法，产生候选集，然后基于候选集选出最优结果。
    
- 首先设定一个候选集大小为k
- 每一步选择可能性最大的k个路径作为下一步的候选集
- 直到最后结束，从k个候选集中选择可能性最大的路径。

 
 NMT的优缺点
 优点是不需要构建人工特征，置需构建端到端的网络
 不足
 
 缺点：
 NMT的解释性差，难以调试，难以控制
    
    
 文献
 【1】CS224n笔记[7 ]:整理了12小时，只为让你20分钟搞懂Seq2seq https://zhuanlan.zhihu.com/p/147310766
    

In [2]:
with open("cmn.txt", "r", encoding="utf-8") as f:
    data = f.read()

In [94]:
# 预处理数据
en = []
cn = []
for x in data.split("\n"):
    if len(x.split("\t")) < 2:
        continue
    ei, ci = x.split("\t")
    ei = ei.lower()
    ei = re.sub(r"([.?,])", r" \1", ei)
    
    en.append(re.split(r"\s", ei))
    cn.append([c for c in ci])
    
    
    

In [109]:
en_word2id = {"<start>": 1, "<pad>": 0}
cn_word2id = {"<end>": 1, "<start>": 2, "<pad>": 0}

In [110]:
input_en = []
for ei in en:
    input_en.append(ei)
    
    for e in ei:
        if e not in en_word2id:
            en_word2id[e] = len(en_word2id)

In [111]:
input_cn = []
target_cn = []
for ci in cn:
    input_cn.append(["<start>"]+ci)
    target_cn.append(ci+["<end>"])
    
    for c in ci:
        if c not in cn_word2id:
            cn_word2id[c] = len(cn_word2id) 

In [112]:
input_en_id = [[en_word2id[e] for e in ei] for ei in input_en]
input_cn_id = [[cn_word2id[c] for c in ci] for ci in input_cn]
target_cn_id = [[cn_word2id[c] for c in ci] for ci in target_cn]

In [113]:
input_en_id_pad = tf.keras.preprocessing.sequence.pad_sequences(input_en_id, maxlen=64, padding="post")
input_cn_id_pad = tf.keras.preprocessing.sequence.pad_sequences(input_cn_id, maxlen=64, padding="post")
target_cn_id_pad = tf.keras.preprocessing.sequence.pad_sequences(target_cn_id, maxlen=64, padding="post")

In [114]:
dataset = tf.data.Dataset.from_tensor_slices((input_en_id_pad, input_cn_id_pad, target_cn_id_pad))

In [115]:
dataset = dataset.shuffle(100).batch(100)

In [85]:
re.split(r"[\s]", en[10])

['He', 'ran', '']

In [123]:
EMBEDING_SIZE = 10
EN_VOCAB_SIZE = len(en_word2id)
CN_VOCAB_SIZE = len(cn_word2id)

VOCAB_SIZE = 10
LSTM_SIZE = 10

In [124]:
class Encoder(tf.keras.Model):
    
    def __init__(self):
        super(Encoder, self).__init__()
        self.word_embed = tf.keras.layers.Embedding(EN_VOCAB_SIZE, EMBEDING_SIZE)
        self.lstm = tf.keras.layers.LSTM(LSTM_SIZE, return_sequences=False, return_state=True)
        
    def call(self, input_x):
        x = self.word_embed(input_x)
        
        x, h_state, c_state = self.lstm(x)
        
        return x, h_state, c_state
        

In [143]:
class Decoder(tf.keras.Model):
    
    def __init__(self):
        super(Decoder, self).__init__()
        self.word_embed = tf.keras.layers.Embedding(CN_VOCAB_SIZE, EMBEDING_SIZE)
        self.lstm = tf.keras.layers.LSTM(LSTM_SIZE, return_sequences=True, return_state=True)
        self.out = tf.keras.layers.Dense(CN_VOCAB_SIZE, activation="softmax")
        
    def call(self, input_x, input_state):
        x = self.word_embed(input_x)
        output, h_state, c_state = self.lstm(x, initial_state=input_state)
        
        logits = self.out(output)
        
        return logits, h_state, c_state

In [126]:
encoder = Encoder()

In [127]:
(x, y, z) = encoder(tf.constant([[1, 2]]))

In [144]:
decoder = Decoder()

In [129]:
x, y, z = decoder(tf.constant([[1, 2]]), [y, x])

In [130]:
optimizer = tf.keras.optimizers.Adam()

In [131]:
loss_fun = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [76]:
x.shape

TensorShape([1, 2, 10])

In [132]:
loss_fun(tf.constant([[1, 1]]), x)

<tf.Tensor: id=4800, shape=(), dtype=float32, numpy=8.144191>

In [133]:
def loss_value(true_val, pred_val):
    return loss_func(true_val, pred_val)

In [148]:
@tf.function
def train_step(input_x, input_y, target_y):
    
    with tf.GradientTape() as tape:
        
    
        _, h_state, c_state = encoder(input_x)
    
        out, h, c = decoder(input_y, [h_state, c_state])
        
        loss = loss_fun(target_y, out)
        
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return loss
    
    

In [ ]:
epoch = 100
for i in range(epoch):
    
    for batch, (input_x, input_y, target_y) in enumerate(dataset):
        
        loss = train_step(input_x, input_y, target_y)
        
        if batch % 100 == 0:
            print("epoch {0} batch {1} loss {2}".format(i, batch, loss))

epoch 0 batch 0 loss 8.144418716430664
epoch 0 batch 100 loss 8.134079933166504
epoch 0 batch 200 loss 7.446378231048584
epoch 1 batch 0 loss 7.288571834564209
epoch 1 batch 100 loss 7.328464508056641
epoch 1 batch 200 loss 7.39698600769043
epoch 2 batch 0 loss 7.240288257598877
epoch 2 batch 100 loss 7.314799785614014
epoch 2 batch 200 loss 7.390826225280762
epoch 3 batch 0 loss 7.233001232147217
epoch 3 batch 100 loss 7.311596870422363
epoch 3 batch 200 loss 7.390583515167236
epoch 4 batch 0 loss 7.229872226715088
epoch 4 batch 100 loss 7.302859306335449
epoch 4 batch 200 loss 7.384603977203369
epoch 5 batch 0 loss 7.311896800994873
epoch 5 batch 100 loss 7.298801422119141
epoch 5 batch 200 loss 7.391597747802734
epoch 6 batch 0 loss 7.242038726806641
epoch 6 batch 100 loss 7.298707485198975
epoch 6 batch 200 loss 7.3855414390563965
epoch 7 batch 0 loss 7.250176429748535
epoch 7 batch 100 loss 7.297691822052002
epoch 7 batch 200 loss 7.382099628448486
epoch 8 batch 0 loss 7.233062744

In [ ]:
def preprocess_en(input_s):
    input_s = input_s.lower()
    input_s = re.sub(r"([.?,])", r" \1", input_s)
    input_s = re.split("\s", input_s)
    
    input_s_id = []
    for s in input_s:
        input_s_id.append(en_word2id[s])
    
    for i in range()

In [ ]:

def predict(input_x: str):
    
    input_x_id = 